Inputs: HOEP and Load Data

Baseline output to determine yearly cost if no storage is available

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [2]:
## Read in csv file from IESO website, ***fill blank entries with 0*** (temp)
df = pd.read_csv("http://reports.ieso.ca/public/PriceHOEPPredispOR/PUB_PriceHOEPPredispOR_2019.csv").fillna(0)

df.rename(columns=df.iloc[2], inplace=True)         ## Set headers to the proper ones row 4
df = df[3:]
df.reset_index(inplace=True, drop=True)             ## Reset indices to proper values

#df.dropna(inplace=True)


## Convert columns to suitable data types
df['Date'] = pd.to_datetime(df['Date'])
df = df.astype({'Hour':int, 'HOEP':float, 'Hour 1 Predispatch': float, 'Hour 2 Predispatch': float, 'Hour 3 Predispatch':float, 'OR 10 Min Sync':float, 'OR 10 Min non-sync':float, 'OR 30 Min':float})

## Split the date into year, month, day
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df.drop(['Date'], axis=1, inplace=True)

## Rearrange them so they appear at the beginning (not necessary, only intermediate step for you to visualize)
date = ['Year', 'Month', 'Day']
df = df[date + [c for c in df if c not in date]]

## Create new dataframe for final data values
data = pd.DataFrame()

## Iterate through the months of the year specified from CSV file
## and iterate through the hours to get monthly average for that specific hour
for month in range(1,13):
    average = []
    df_month = df.loc[df['Month'] == month]

    for hour in range(1,25):
        h = df_month.loc[df['Hour'] == hour]
        average.append((h['HOEP'].sum()/h.shape[0])/100)  ## Cents to Dollars
    data['Month ' + str(month)] = average

## Set index to proper hours
data.index = range(1, len(data)+1)
data.index.name = 'Hour'


data_6months = pd.DataFrame()
for i in range(0, 12, 2):
    data_6months[data.columns[i] + ' & ' + data.columns[i+1]] = (data[data.columns[i]] + data[data.columns[i+1]])/2
#data_6months

In [3]:
load_data = pd.read_excel('load_data.xlsx', index_col=0)
#load_data

In [4]:
# Monthly Bill 
WMST = 0.003499             ## $/kWh 
KDC = 0.43


In [7]:
## x = hourly cost

x = load_data.to_numpy() * (data_6months.to_numpy()+WMST)
mec = sum(30*sum(x))
mdc = 0 
for i in range(6): 
    mdc = mdc + KDC*30*np.amax(load_data.iloc[:,i].to_numpy())
    
print(2*(mdc+mec))



1987659.6232769976


In [19]:
df_sol = pd.concat([pd.DataFrame(data=x), pd.DataFrame(data=load_data), pd.DataFrame(data=data_6months.to_numpy())], axis=1)
df_sol


,0,1,2,3,4,5,Month 1 & Month 2,Month 3 & Month 4,Month 5 & Month 6,Month 7 & Month 8,Month 9 & Month 10,Month 11 & Month 12,0,1,2,3,4,5
0,298.354827,99.148222,19.142821,113.699646,27.614173,187.118099,1438.395,604.9708,902.2919,1075.939,1075.725,1314.213,0.203923,0.160390,0.017717,0.102176,0.022171,0.138881
1,273.289672,101.425646,-16.907306,82.361911,29.605138,188.041077,1414.594,605.7097,888.1929,1045.151,1034.786,1320.044,0.189694,0.163950,-0.022535,0.075305,0.025111,0.138952
2,257.266477,97.511288,-19.644183,73.945907,22.970500,156.851736,1418.216,605.0158,925.5618,1054.296,1052.425,1300.168,0.177902,0.157672,-0.024723,0.066639,0.018327,0.117141
3,279.653865,91.736589,-14.373185,67.919416,19.658377,161.451324,1432.950,604.2784,907.4609,1024.815,1026.229,1313.003,0.191661,0.148313,-0.019338,0.062776,0.015657,0.119464
4,254.785324,87.733858,-7.920631,65.055109,26.011901,134.147008,1407.696,610.8801,923.2916,1050.769,1032.733,1303.902,0.177496,0.140120,-0.012078,0.058413,0.021688,0.099382
5,248.428345,114.005822,20.132460,89.549182,87.178869,116.280683,1399.250,606.2308,1013.8140,1169.434,1142.472,1320.345,0.174045,0.184558,0.016359,0.073076,0.072808,0.084569
6,314.349482,165.380358,50.476086,152.571372,141.008718,180.099503,1425.513,628.7807,1106.3390,1268.285,1221.761,1344.123,0.217018,0.259519,0.042125,0.116798,0.111915,0.130491
7,552.859999,169.006044,120.709294,234.454063,125.184968,492.776015,1487.829,634.4462,1158.4610,1334.432,1269.521,1403.989,0.368089,0.262885,0.100699,0.172197,0.095109,0.347484
8,459.798010,144.901552,85.599661,263.973551,178.119301,617.044123,1500.000,583.6489,1183.6180,1381.811,1270.018,1467.105,0.303033,0.244769,0.068821,0.187535,0.136750,0.417087
9,501.552869,131.828238,65.226907,284.320081,174.624446,460.265655,1492.180,568.3974,1183.5750,1413.208,1278.483,1460.864,0.332622,0.228431,0.051611,0.197689,0.133088,0.311565


In [20]:
df_sol.columns = load_data.columns.values.tolist() + load_data.columns.values.tolist() + load_data.columns.values.tolist()
df_sol

,Month 1 & Month 2,Month 3 & Month 4,Month 5 & Month 6,Month 7 & Month 8,Month 9 & Month 10,Month 11 & Month 12,Month 1 & Month 2,Month 3 & Month 4,Month 5 & Month 6,Month 7 & Month 8,Month 9 & Month 10,Month 11 & Month 12,Month 1 & Month 2,Month 3 & Month 4,Month 5 & Month 6,Month 7 & Month 8,Month 9 & Month 10,Month 11 & Month 12
0,298.354827,99.148222,19.142821,113.699646,27.614173,187.118099,1438.395,604.9708,902.2919,1075.939,1075.725,1314.213,0.203923,0.160390,0.017717,0.102176,0.022171,0.138881
1,273.289672,101.425646,-16.907306,82.361911,29.605138,188.041077,1414.594,605.7097,888.1929,1045.151,1034.786,1320.044,0.189694,0.163950,-0.022535,0.075305,0.025111,0.138952
2,257.266477,97.511288,-19.644183,73.945907,22.970500,156.851736,1418.216,605.0158,925.5618,1054.296,1052.425,1300.168,0.177902,0.157672,-0.024723,0.066639,0.018327,0.117141
3,279.653865,91.736589,-14.373185,67.919416,19.658377,161.451324,1432.950,604.2784,907.4609,1024.815,1026.229,1313.003,0.191661,0.148313,-0.019338,0.062776,0.015657,0.119464
4,254.785324,87.733858,-7.920631,65.055109,26.011901,134.147008,1407.696,610.8801,923.2916,1050.769,1032.733,1303.902,0.177496,0.140120,-0.012078,0.058413,0.021688,0.099382
5,248.428345,114.005822,20.132460,89.549182,87.178869,116.280683,1399.250,606.2308,1013.8140,1169.434,1142.472,1320.345,0.174045,0.184558,0.016359,0.073076,0.072808,0.084569
6,314.349482,165.380358,50.476086,152.571372,141.008718,180.099503,1425.513,628.7807,1106.3390,1268.285,1221.761,1344.123,0.217018,0.259519,0.042125,0.116798,0.111915,0.130491
7,552.859999,169.006044,120.709294,234.454063,125.184968,492.776015,1487.829,634.4462,1158.4610,1334.432,1269.521,1403.989,0.368089,0.262885,0.100699,0.172197,0.095109,0.347484
8,459.798010,144.901552,85.599661,263.973551,178.119301,617.044123,1500.000,583.6489,1183.6180,1381.811,1270.018,1467.105,0.303033,0.244769,0.068821,0.187535,0.136750,0.417087
9,501.552869,131.828238,65.226907,284.320081,174.624446,460.265655,1492.180,568.3974,1183.5750,1413.208,1278.483,1460.864,0.332622,0.228431,0.051611,0.197689,0.133088,0.311565


In [ ]:
df_sol.to_excel('Opt_Module_vBaseline.xlsx')

In [ ]:
p = sol.x[0:24]

In [ ]:
e = sol.x[24:48]

In [ ]:
x_ax = np.arange(1,25)
x_ax

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axs = plt.subplots(1,3)
fig.set_size_inches(20, 5)
axs[0].plot(x_ax, p)
axs[0].plot(x_ax, e)
axs[1].plot(x_ax, data_6months.iloc[:,0])
axs[0].legend(['PCEA', 'ESB'])
axs[2].plot(x_ax, load_data.iloc[:,0])

In [ ]:
import scipy
scipy.__version__